In [2]:
import os

from IPython import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_io as tfio

AttributeError: module 'tensorflow._api.v2.compat.v2.__internal__' has no attribute 'monitoring'

In [ ]:
items = pd.read_csv("../Datasets/ml-100k/Text/items.csv")

In [ ]:
yamnet_model_handle = 'https://tfhub.dev/google/yamnet/1'
yamnet_model = hub.load(yamnet_model_handle)

In [ ]:
# testing_wav_file_name = tf.keras.utils.get_file('miaow_16k.wav',
#                                                 'https://storage.googleapis.com/audioset/miaow_16k.wav',
#                                                 cache_dir='./',
#                                                 cache_subdir='test_data')

# print(testing_wav_file_name)

In [ ]:
@tf.function
def load_wav_16k_mono(filename):
    """ Load a WAV file, convert it to a float tensor, resample to 16 kHz single-channel audio. """
    file_contents = tf.io.read_file(filename)
    wav, sample_rate = tf.audio.decode_wav(
          file_contents,
          desired_channels=1)
    wav = tf.squeeze(wav, axis=-1)
    sample_rate = tf.cast(sample_rate, dtype=tf.int64)
    wav = tfio.audio.resample(wav, rate_in=sample_rate, rate_out=16000)
    return wav

In [ ]:
# testing_wav_data = load_wav_16k_mono(testing_wav_file_name)

# _ = plt.plot(testing_wav_data)

# # Play the audio file.
# display.Audio(testing_wav_data,rate=16000)

In [ ]:
# class_map_path = yamnet_model.class_map_path().numpy().decode('utf-8')
# class_names =list(pd.read_csv(class_map_path)['display_name'])

# for name in class_names[:20]:
#   print(name)
# print('...')

In [ ]:
def get_embeddings(testing_wav_data):
    scores, embeddings, spectrogram = yamnet_model(testing_wav_data)
#     class_scores = tf.reduce_mean(scores, axis=0)
#     top_class = tf.argmax(class_scores)
#     inferred_class = class_names[top_class]
    return (np.mean(embeddings, axis = 0))
#     print(np.mean(embeddings, axis = 0))
#     print(f'The main sound is: {inferred_class}')
#     print(f'The embeddings shape: {embeddings.shape}')

In [ ]:
import os
embeddings=[]
for index, movie in items.iterrows():
    #print(index)
    print(movie['YT-Trailer ID'])
    path = "../Datasets/ml-100k/Audio/{}.wav".format(movie['YT-Trailer ID'])
    print(path)
#     if(movie['YT-Trailer ID'] in embeddings.columns):
#         print("hello")
#         continue
    if(os.path.exists(path) == False):
        print("hello1")
        continue
    elif(os.path.getsize(path)>=100000000):
        continue
    else:
        print(os.path.getsize(path))
        #print(get_embeddings(load_wav_16k_mono(path)))
        embeddings.append(get_embeddings(load_wav_16k_mono(path)))
        #print(embeddings)
        print("Updating")
    #print(index)
    pd.DataFrame(embeddings).to_csv("../Datasets/ml-100k/Audio/embeddings.csv",index=False)
#     if(index % 10 == 0):
    